In [1]:
import numpy as np
import pandas as pd
import lmdb
import caffe

In [2]:
df = pd.read_hdf('sensormatrix.h5')
df2 = pd.read_hdf('referencematrix.h5')
X = df.values
Y = df2.values

In [3]:
X_train = X[:,:60000]
Y_train = Y[:,:60000]
X_test = X[:,60000:]
Y_test = Y[:,60000:]

In [4]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

In [9]:
map_size = X.nbytes*10
env_x = lmdb.open('sensormatrix_train_x_lmdb_2', map_size=map_size) # you can put map_size a little bigger 
env_y = lmdb.open('sensormatrix_train_y_lmdb_2', map_size=map_size)


In [ ]:
with env_x.begin(write=True) as txn_x, env_y.begin(write=True) as txn_y:
    for i in xrange(X_train.shape[1]):
        x = X_train[:,i]
        y = Y_train[:,i] 

        datum_x = caffe.io.array_to_datum(arr=x.reshape((60,1,1)),label=i)
        datum_y = caffe.io.array_to_datum(arr=y.reshape((60,1,1)),label=i)
        keystr = '{:0>10d}'.format(i) # format an lmdb key for this entry
        txn_x.put( keystr, datum_x.SerializeToString() ) # actual write to lmdb
        txn_y.put( keystr, datum_y.SerializeToString() )


In [12]:
map_size = X.nbytes*10
envt_x = lmdb.open('sensormatrix_test_x_lmdb_2', map_size=map_size) # you can put map_size a little bigger 
envt_y = lmdb.open('sensormatrix_test_y_lmdb_2', map_size=map_size)

In [13]:
with envt_x.begin(write=True) as txn_x, envt_y.begin(write=True) as txn_y:
    for i in xrange(X_test.shape[1]):
        x = X_test[:,i]
        y = Y_test[:,i] 

        datum_x = caffe.io.array_to_datum(arr=x.reshape((60,1,1)),label=i)
        datum_y = caffe.io.array_to_datum(arr=y.reshape((60,1,1)),label=i)
        keystr = '{:0>10d}'.format(i) # format an lmdb key for this entry
        txn_x.put( keystr, datum_x.SerializeToString() ) # actual write to lmdb
        txn_y.put( keystr, datum_y.SerializeToString() )


In [7]:
import h5py